In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [2]:
import numpy as np
import cv2
import pandas as pd
from tqdm.notebook import tqdm 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision 
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import time

Using plaidml.keras.backend backend.


In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
original_fake_paths = []

for dirname, _, filenames in tqdm(os.walk('/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/1-million-fake-faces/')):
    for filename in filenames:
        original_fake_paths.append([os.path.join(dirname, filename), filename])

In [5]:
original_fake_paths.remove(original_fake_paths[0])

In [6]:
original_fake_paths

[['/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/1-million-fake-faces/1m_faces_09/XIY8B3C6RH.jpg',
  'XIY8B3C6RH.jpg'],
 ['/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/1-million-fake-faces/1m_faces_09/63KHTTUD1X.jpg',
  '63KHTTUD1X.jpg'],
 ['/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/1-million-fake-faces/1m_faces_09/WH3L7SF893.jpg',
  'WH3L7SF893.jpg'],
 ['/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/1-million-fake-faces/1m_faces_09/DWN1FDAUBR.jpg',
  'DWN1FDAUBR.jpg'],
 ['/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/1-million-fake-faces/1m_faces_09/LEBMKHNQ0D.jpg',
  'LEBMKHNQ0D.jpg'],
 ['/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/1-million-fake-faces/1m_faces_09/NEV0JH5MT0.jpg',
  'NEV0JH5MT0.jpg'],
 ['/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/1-million-fake-faces/1m_faces_09/VO0U7X9HMM.jpg',
  'VO0U7X9HMM.jpg'],
 ['/Users/rahultorlapati/Downloads/Semester 4/DATA 606/

In [7]:
save_dir = '/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/tmp/fake/'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [8]:
fake_paths = [save_dir + filename for _, filename in original_fake_paths]

In [9]:
'''for path, filename in tqdm(original_fake_paths):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    cv2.imwrite(os.path.join(save_dir, filename), img)'''

'for path, filename in tqdm(original_fake_paths):\n    img = cv2.imread(path)\n    img = cv2.resize(img, (224, 224))\n    cv2.imwrite(os.path.join(save_dir, filename), img)'

In [10]:
train_fake_paths, test_fake_paths = train_test_split(fake_paths, test_size=20000, random_state=2019)

fake_train_df = pd.DataFrame(train_fake_paths, columns=['filename'])
fake_train_df['class'] = 'FAKE'

fake_test_df = pd.DataFrame(test_fake_paths, columns=['filename'])
fake_test_df['class'] = 'FAKE'

In [11]:
real_dir = '/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/celeba-dataset/img_align_celeba/img_align_celeba/'
eval_partition = pd.read_csv('/Users/rahultorlapati/Downloads/Semester 4/DATA 606/Datasets/celeba-dataset/list_eval_partition.csv')

eval_partition['filename'] = eval_partition.image_id.apply(lambda st: real_dir + st)
eval_partition['class'] = 'REAL'

In [12]:
real_train_df = eval_partition.query('partition in [0, 1]')[['filename', 'class']]
real_test_df = eval_partition.query('partition == 2')[['filename', 'class']]

In [13]:
train_df = pd.concat([real_train_df[0:5000], fake_train_df[0:5000]])
test_df = pd.concat([real_test_df[0:5000], fake_test_df[0:5000]])

In [14]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_dataframe(
    train_df,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_dataframe(
    train_df,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
    subset='validation'
)

Found 4000 validated image filenames belonging to 1 classes.
Found 1000 validated image filenames belonging to 1 classes.


/opt/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 5000 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [15]:
datagen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    test_df,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

Found 5000 validated image filenames belonging to 1 classes.


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(224,224,3)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

INFO:plaidml:Opening device "metal_amd_radeon_pro_5500m.0"


In [17]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 32)      18464     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1548800)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 3097602   
Total params: 3,117,858
Trainable params: 3,117,858
Non-trainable params: 0
_________________________________________________________________


In [20]:
checkpoint = ModelCheckpoint('model.h5', save_best_only=True)

train_history_step1 = model.fit_generator(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint],
    epochs=7
)

Epoch 1/7
63/63 [==============================] - 130s 2s/step - loss: 0.0110 - acc: 0.9926 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/7
63/63 [==============================] - 127s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 3/7
63/63 [==============================] - 127s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 4/7
63/63 [==============================] - 127s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 5/7
63/63 [==============================] - 127s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 6/7
63/63 [==============================] - 127s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 7/7
63/63 [==============================] - 127s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000


In [21]:
model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(224,224,3)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(2, activation='softmax'))

In [22]:
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 774400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               99123328  
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [24]:
checkpoint = ModelCheckpoint('model1.h5', save_best_only=True)

train_history_step2 = model1.fit_generator(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint],
    epochs=7
)

Epoch 1/7
63/63 [==============================] - 130s 2s/step - loss: 0.0112 - acc: 0.9928 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/7
63/63 [==============================] - 127s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 3/7
63/63 [==============================] - 126s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 4/7
63/63 [==============================] - 126s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 5/7
63/63 [==============================] - 126s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 6/7
63/63 [==============================] - 126s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 7/7
63/63 [==============================] - 126s 2s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
